Starting with creating a CLI version of my program.

Starting off with creating a simple version of my program that can run with csv files. Does not take into account draft position or how many players your team can fit.

In [37]:
pip install sleeper-py
# found this resource on the internet when looking for sleeper api python on accident, decided to try it out

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import packages
import pandas as pd
import requests
from pprint import pprint
import json
import csv

In [2]:
# constants
API = "https://api.sleeper.app/v1/players/nfl"

Starting with actual code

In [3]:
# get response from api request
response = requests.get(API)
display(response)
print(response.url)

<Response [200]>

https://api.sleeper.app/v1/players/nfl


In [4]:
def draft(player_name, player_dict, user_draft):
    '''finds player and add to list of user's players
    params: player_name (string)
            player_dict (dict)
            user_draft (list)
    returns: user_draft
    '''
    found_player = False
    for key in player_dict: # looking for player by iterating through dictionary
        player = player_dict[key]
        full_name = (player['first_name'] + " " + player['last_name']).lower()
        if full_name == player_name.lower(): # player is only appended if name in dictionary is same
            user_draft.append(player)
            print(f"you have drafted {player_name}")
            found_player = True

    if found_player == False: # returns false if player not found
        print(f"player {player_name} not found")
        return False
    else:
        return user_draft

In [5]:
def predicted_spots(position, league_size, team_size):
    '''def predicted_spots returns all predicted draft spots the user will draft at
    
    params: position (string): position that player's first pick is
            league_size (string): number of people in the league
            team_size (string): number of players on each team
    '''
    draft_order = [] # list for all picks
    current_pick = int(position) # starting at the first pick
    
    for round_number in range(int(team_size)):
        if round_number % 2 == 0: # regular order for odd rounds
            current_round = list(range(1, int(league_size) + 1))
        else: # reverse order for even rounds (snake draft)
            current_round = list(range(int(league_size), 0, -1))
        
        # calculate total pick number
        draft_order.append(current_pick + current_round[int(position) - 1] - 1)
        current_pick += int(league_size)
    
    return draft_order

In [6]:
# testing predicted_spots
print(predicted_spots("1", "12", "15"))

[1, 24, 25, 48, 49, 72, 73, 96, 97, 120, 121, 144, 145, 168, 169]


In [7]:
def calculate_grade(user_draft, draft_positions):
    '''
    def calculate_grade calculates draft grade by predicted ranking

    params: user_draft (list): list of players user has drafted
            draft_positions (list): list of positions user is drafting in
    returns: grade (string): letter grade assigned to value, prints predicted score and your score as well
    '''

    predicted_score = 0 # predicted score is based on player ranking from previous seasons, by adding all the picks we should get the predicted score as the predicted draft will be the player rated at that draft pick (i.e. the 72nd pick will be the 72nd best player)
    for round in draft_positions:
        predicted_score += round
    
    print(f"your predicted score is: {predicted_score}")

    score = 0
    for player in user_draft:
        score += player['search_rank']

    print(f"your score is: {score}") # printing both predicted and real score to let user see a little on how it was graded

    print("your draft grade is: ")
    if score <= predicted_score - 75:
        return "S"
    if score <= predicted_score - 50:
        return "A"
    if score <= predicted_score - 25:
        return "B"
    if score <= predicted_score or score <= predicted_score + 25:
        return "C"
    if score <= predicted_score + 50:
        return "D"
    else:
        return "F"
    
    # overall this scoring part needs work but i spent the last two weeks drawing up ways on how i can do it better to no avail. this will be worked on for v2

In [8]:
def read_players(filename):
    '''def read_players reads in players into json file

    params: filename (string): filename to read
    returns: player_dict (dictionary): dictionary with all players
    '''
    with open(filename , "r") as fo:
        player_dict = json.load(fo) 
    return player_dict

In [ ]:
def read_csv(filename):
    with open(filename, 'r') as csv_file:
        # creating csv reader
        csv_reader = csv.DictReader(csv_file)

        # creating list to store json objects
        json_data = []

        for row in csv_reader:
            json_data.append(row)

    

In [ ]:
user_draft = []
player_dict = read_players("players.json")

league_size = input("how many people are in your league? (input int, not string)")
team_size = input("how many players are on each team? (input int, not string)")
position = input("what number pick is your first pick? (input int, not string)")

draft_positions = predicted_spots(position, league_size, team_size)

x = 0 # counter for how many picks
while x < int(team_size): # user picks team
    player_name = input("you are currently picking at pick " + str(draft_positions[x]) + ". please input your player's full name!")

    drafted = draft(player_name, player_dict, user_draft) # originally this was in the if statement but it was printing twice so i fixed by doing this

    if drafted == False:
        print("try again!")
    else:
        user_draft = drafted
        x += 1

print(calculate_grade(user_draft, draft_positions))

player  not found
try again!
player  not found
try again!
player  not found
try again!
player  not found
try again!
player  not found
try again!
player  not found
try again!
player  not found
try again!
player  not found
try again!
player  not found
try again!
player  not found
try again!


Creating GUI

In [1]:
%pip install streamlit

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   --- ------------------------------------ 0.8/8.7 MB 5.6 MB/s eta 0:00:02
   -------- ------------------------------- 1.8/8.7 MB 7.2 MB/s eta 0:00:01
   ------------------------- -------------- 5.5/8.7 MB 10.2 MB/s eta 0:00:01
   ----------------------------------- ---- 7.9/8.7 MB 10.8 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 10.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/658.1 kB ? eta -:--:--
   --------------------------------------- 658.1/658.1 kB 12.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   --- ------------------------------------ 2.1/25.1 MB 10.7 MB/s eta 0:00:03
   ------ --------------------------------- 4.2/25.1 MB 10.5 MB/s eta 0:00:02
   --------- --------------


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\hyunsoo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import streamlit as st

In [5]:
# just trying to create a streamlit application, import of backend will come in part 2b

st.write("hello world")

2024-11-05 00:51:05.338 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 00:51:05.729 
  command:

    streamlit run C:\Users\hyunsoo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-11-05 00:51:05.729 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 00:51:05.730 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-05 00:51:05.730 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
